In [32]:
from pdf2image import convert_from_path

In [33]:
# pdf2image takes the pdf file as well as the path to the prequiisite which it needs to function 
# and converts the file to an Image. The image here is not very clear that is why we need a OpenCV 
pages = convert_from_path(r'docs\prescription\pre_1.pdf', poppler_path=r'C:\poppler-22.04.0\Library\bin')

In [3]:
pages[0].show()

In [34]:
import pytesseract

In [5]:
# pytesseract tkes the image and extract text from it but it missed some text in this case. 
# so we need OPenCV to give clear image before we extract using tesseract
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Maria Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

—momennannenncmneneunnmnnnnninsissiyoinnitnahaadaanih issn earnttneenrenen:

Prednisone 20 mg
Lialda 2.4 gram

3 days,

or 1 month


In [35]:
#To get a clear image we will create a function which use OPenCV adaptive thressholding for our image
# Simple thressholding will not be good enough for us as explained in the file cv_concept
import numpy as np                   #module that will convert image to array
import cv2                           #module that will create a clear image
from PIL import Image                #module that will display the picture

#we create a function that will perform adaptive thressholding. CHECK GOOLGE FOR SYNTAX
def preprocess_image(img):
 # below we take the image file suplied by user and convert it to grey using the cv2.cvtColor function. Check google for syntax
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR) #resize the grey image
# Use cv2 adaptive method to get a clear picture
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

In [36]:
# NOW we will use our function (preprocess_image) on the pages[0] image which we got by using pdf2image
# The function is going to produce clear image
img = preprocess_image(pages[0])
Image.fromarray(img).show()              #this module will show the image of the resulting array

In [37]:
#Now pytesseract can now extract correctly and clearly
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(img, lang='eng')
print(text)

Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222

Name: Marta Sharapova Date: 5/11/2022

Address: 9 tennis court, new Russia, DC

K

Prednisone 20 mg
Lialda 2.4 gram

Directions:

Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks a
Lialda - take 2 pill everyday for 1 month

Refill: 2 times


### We Now extract important information from the extracted text


In [38]:
text = '''
Dr John Smith, M.D
2 Non-Important Street,
New York, Phone (000)-111-2222
Name: Marta Sharapova Date: 5/11/2022
Address: 9 tennis court, new Russia, DC

 
 

Prednisone 20 mg
Lialda 2.4 gram
Directions:
Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks -
Lialda - take 2 pill everyday for 1 month
Refill: 3 times
'''

In [39]:
import re
pattern = 'Address:(.*)\n'

matches = re.findall(pattern, text)
matches[0].strip()

'9 tennis court, new Russia, DC'

In [41]:
pattern = 'Address:(.*)'

matches = re.findall(pattern, text)
matches[0].strip()

'9 tennis court, new Russia, DC'

In [42]:
import re

pattern = 'Name:(.*)Date'

matches = re.findall(pattern, text)
matches[0].strip()

'Marta Sharapova'

In [43]:
pattern = 'Address[^\n]*(.*)Directions'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Prednisone 20 mg\nLialda 2.4 gram'

In [44]:
pattern = 'Directions:(.*)Refill:'
matches = re.findall(pattern, text, flags=re.DOTALL)
print(matches[0].strip())

Prednisone, Taper 5 mg every 3 days,
Finish in 2.5 weeks -
Lialda - take 2 pill everyday for 1 month


In [45]:
pattern = 'Directions:(.*)Refill'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Prednisone, Taper 5 mg every 3 days,\nFinish in 2.5 weeks -\nLialda - take 2 pill everyday for 1 month'

In [46]:
pattern = 'Refill:(.*)times'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'3'

In [47]:
pattern = 'Refill: (\d+) times'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'3'

In [48]:
#Extract all Twitter handles from the following text (i.e. your answer should be teslarati, dummy_tesla, and dummy_2_tesla). 
#Twitter handle is the text that appears after https://twitter.com/ and is a single word. Also, 
#it contains only alphanumeric characters i.e. A-Z a-z, o to 9, and underscore _

text = '''
Follow our leader Elon musk on Twitter 
here: https://twitter.com/elonmusk, more information 
on Tesla's products can be found at https://www.tesla.com/. 
Also here are leading influencers for tesla-related news, 
https://twitter.com/teslarati
https://twitter.com/dummy_tesla
https://twitter.com/dummy_2_tesla'''

In [31]:
pattern = re.compile(r'https:\/\/twitter.com\/([a-zA-Z0-9_]+)')
matches = pattern.findall(text)
matches

['elonmusk', 'teslarati', 'dummy_tesla', 'dummy_2_tesla']